In [ ]:
name_layer = 'conv5'

@tf.custom_gradient
def guided_Relu(x):
  def grad_1(dy):
    return tf.cast(dy>0,"float32") * tf.cast(x>0, "float32") * dy
  return tf.nn.relu(x), grad_1



def back_prop(model, layer=name_layer, N_new=10, cat = 0,fil = 2):
  gb_model = tf.keras.models.Model(
    inputs = [model.inputs],    
    outputs = [model.get_layer(name_layer).output])
  layer_dict = [layer for layer in gb_model.layers[1:] if hasattr(layer,'activation')]

  for layer in layer_dict:
    if layer.activation == tf.keras.activations.relu:
      layer.activation = guided_Relu
  category = categories[cat]
  dir=os.path.join(train_dir,category)
  fig = plt.figure(figsize=(30,20))
  for img in os.listdir(dir):
    columns=5
    rows = 2
    
    img_path=(os.path.join(dir,img))
    imgs=cv2.imread(img_path)
    imgs=mpimg.imread(img_path)
    new_img=cv2.resize(imgs,(300,300))
    x = np.expand_dims(new_img, axis=0)
    j=1
    i=0
    while j <= N_new:
      with tf.GradientTape() as tape:
        inputs = tf.cast(x, tf.float32)
        tape.watch(inputs)
        outputs = gb_model(inputs)[0]
        
      
        
        if outputs[i,i,fil] != 0:
          out = outputs[i,i,fil]
          grads = tape.gradient(out,inputs)[0]
          guided_back_prop =grads
          gb_viz = np.dstack((
                      guided_back_prop[:, :, 0],
                      guided_back_prop[:, :, 1],
                      guided_back_prop[:, :, 2],
                  ))       
          gb_viz -= np.min(gb_viz)
          gb_viz /= gb_viz.max()
          fig.add_subplot(rows,columns,j)
          plt.imshow(gb_viz)
          plt.title(category)
          j+=1
          i+=1
        else:
          i+=1  
    break

In [ ]:
back_prop(model)